# Simulação de Monte Carlo

* Criada durante a segunda guerra mundial;
* Nomeado em homenagem a cidade de Mônaco com seus cassinos;
* Foi criada originalmente para testar cenários de um evento incerto. O acaso é o principal elemento da abordagem na modelagem.

* No mercado financeiro, existem tantos cenários, que a projeção de retornos não pode ser resolvida por uma simples análise combinatória. É necessário resolver o problema utilizando força bruta.
* Simular o cenário **N** vezes, criando uma amostra e, a partir da amostra, praticar a inferência estatística.

# Problema a ser resolvido:

* Tenho uma carteira com 10 ações. Dado a distribuição dos retornos dessa carteira no último ano:
    * Qual o máximo que eu posso perder nos próximos 3 anos, com uma confiança de 95%?
    * Qual a probabilidade de eu obter lucro?

# Resolução:

* Simular 10 mil cenários da carteira com 10 ações nos próximos 10 anos e criar uma distribuição dos retornos com os cenários. Para isso, vamos utilizar Monte Carlo.

## 1.0 Bibliotecas

In [2]:
import numpy                            as np
import matplotlib.pyplot                as plt
import datetime                         as dt
from pandas_datareader   import data    as pdr
from numpy               import linalg  as LA

## 2. Obtendo os dados

In [3]:
lista_acoes = ['WEGE3', 'PCAR3', 'LREN3', 'PETR4', 'VALE3'] # Lista de ativos de interesse
lista_acoes = [acao + '.SA' for acao in lista_acoes] # Adicionando o sufixo '.SA' para consultar as cotações no Yahoo Finance

data_final = dt.datetime.now() # Definindo a data final como a data atual (hoje)
data_inicial = data_final - dt.timedelta(days=300) # A data final é 300 dias para trás

precos = pdr.get_data_yahoo(lista_acoes, data_inicial, data_final)['Adj Close'] # Pegar o valor da cotação de fechamento AJUSTADO

## 3. Calculando os retornos da matriz de Covariância

In [ ]:
retornos = precos.pct_change().dropna() # dropna, pois a primeira linha não haverá percentual
media_retornos = retornos.mean() #calcula a média dos retornos
matriz_covariancia = retorno.cov() # calcular a matriz de covariância
pesos_carteira = np.full(len(lista_acoes), 1/len(lista_acoes)) # calcular o peso da carteira dividindo o tamanho da carteira (100%) pelo número de ativos
numero_acoes = len(lista_acoes)

pesos_carteira

## 4. Fórmula dos Retornos Sintéticos

Retornos sintéticos = media_retornos + Rpdf x L

* media_retornos = Média dos retornos
* Rpdf = Matriz aleatória gerada por alguma função de densidade de probabilidade
* L = Matriz triangular inferior proveniente de uma decomposição de Cholesky, usando como base a matriz de covariância dos dados originais.

**Explicação:**

* Assumimos que a distribuição dos retornos se aproxima de uma normal.<br>

* Ao gerar retornos aleatórios de cada ativo, estes valores compõem vetores aleatórios descorrelacionados. Para corrigir isso, usaremos uma matriz triangular obtida a partir da covariância. 

## 5. Premissas do Modelo de Monte Carlo

In [ ]:
numero_simulacoes = 10000 # número de simulações
dias_projetados = 252 * 3 # 3 anos com 252 dias de bolsa
capital_inicial = 1000 # capital inicial

# retorno médio
retorno_medio = retornos.mean(axis=0).to_numpy()
matriz_retorno_medio = retorno_medio * np.ones(shape = (dias_projetados, numero_acoes))

# Gerando o valor de L
L = LA.cholesky(matriz_covariancia)
L

In [ ]:
# Gerando simulações

retornos_carteira = np.zeros([dias_projetados, numero_simulacoes]) # cada coluna é uma simulacao
montante_final = np.zeros(numero_simulacoes)

for s in range(numero_simulacoes):
    Rpdf = np.random.normal(size=(dias_projetados, numero_acoes))
    retornos_sinteticos = matriz_retorno_medio + np.inner(Rpdf, L)
    
    retornos_carteira[:, s] = np.cumprod(np.inner(pesos_carteira, retornos_sinteticos)+1)*capital_inicial
    montante_final[s] = retornos_carteira[-1, s]

montante_final

In [ ]:
plt.plot